In [1]:
from pyspark import SparkContext

# Initialize SparkContext
sc = SparkContext(master="local", appName="RDD")

In [2]:

sc

<SparkContext master=local appName=RDD>

builder a rdd from  a python collection 


In [3]:
distData = sc.parallelize([1,2,3,4])
distData

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [4]:
distData.collect()

[1, 2, 3, 4]

Basic statistics using RDD of the number 

In [5]:
from random import random

In [6]:
l =[random() for _ in range(10)]
l

[0.12781500195197537,
 0.6980680756398487,
 0.777692227558757,
 0.8997286898973403,
 0.6822916532972078,
 0.5979234897699601,
 0.9441936264745765,
 0.9488217034125386,
 0.22023594736876062,
 0.0352668414357703]

In [7]:
rdd =sc.parallelize(l,numSlices=4) #number of partition 4

In [8]:
rdd.max() , rdd.sum()

(0.9488217034125386, 5.932037256806735)

In [9]:
rdd.stats()

(count: 10, mean: 0.5932037256806735, stdev: 0.3261276251875564, max: 0.9488217034125386, min: 0.0352668414357703)

Exercise 1:

Create a directory in HDFS called movielens and load movies.csv and ratings.csv from ~/Downloads/datasets/movie-lens directory.
Load movies.csv into in moviesRdd
Load ratings.csv into ratingsRdd
Find out 10 top movies based on the highest average rating. Consider only those movies that have got at least 100 ratings.

In [10]:
 ls data/movieslens/*

data/movieslens/links.csv    data/movieslens/ratings.csv
data/movieslens/movies.csv   data/movieslens/ratings.json
data/movieslens/movies.json  data/movieslens/tags.csv


In [11]:
movies = sc.textFile("data/movieslens/movies.csv")
movies.count()

9126

In [12]:
movies.top(10)

['movieId,title,genres',
 '99992,Shadow Dancer (2012),Crime|Drama|Thriller',
 '99917,Upstream Color (2013),Romance|Sci-Fi|Thriller',
 '99912,Mama (2013),Horror',
 '999,2 Days in the Valley (1996),Crime|Film-Noir',
 '99846,Everything or Nothing: The Untold Story of 007 (2012),Documentary',
 '99839,Paul Williams Still Alive (2011),Comedy|Documentary|Musical',
 '99813,"Batman: The Dark Knight Returns, Part 2 (2013)",Action|Animation',
 '99811,Beware of Mr. Baker (2012),Documentary',
 '998,Set It Off (1996),Action|Crime']

In [13]:
for _ in movies.take(10):
    print(_)

movieId,title,genres
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
6,Heat (1995),Action|Crime|Thriller
7,Sabrina (1995),Comedy|Romance
8,Tom and Huck (1995),Adventure|Children
9,Sudden Death (1995),Action


*top():* Retrieves the top elements from an RDD based on a specified ordering.

*take():* Retrieves the first elements from an RDD based on their position without any specific ordering.

In [15]:
moviesData = movies.filter(lambda line: not line.startswith("movieId"))
for r in moviesData.take(10):
    print(r)

1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
6,Heat (1995),Action|Crime|Thriller
7,Sabrina (1995),Comedy|Romance
8,Tom and Huck (1995),Adventure|Children
9,Sudden Death (1995),Action
10,GoldenEye (1995),Action|Adventure|Thriller


In [16]:
ratings = sc.textFile("data/movieslens/ratings.csv")
ratings.count()

100005

In [17]:
for r in ratings.take(10):
    print(r)

userId,movieId,rating,timestamp
1,31,2.5,1260759144
1,1029,3.0,1260759179
1,1061,3.0,1260759182
1,1129,2.0,1260759185
1,1172,4.0,1260759205
1,1263,2.0,1260759151
1,1287,2.0,1260759187
1,1293,2.0,1260759148
1,1339,3.5,1260759125


In [18]:
ratingsData = ratings.filter(lambda line: not line.startswith("userId"))
for _ in ratingsData.take(10): 
    print(_)

1,31,2.5,1260759144
1,1029,3.0,1260759179
1,1061,3.0,1260759182
1,1129,2.0,1260759185
1,1172,4.0,1260759205
1,1263,2.0,1260759151
1,1287,2.0,1260759187
1,1293,2.0,1260759148
1,1339,3.5,1260759125
1,1343,2.0,1260759131


In [19]:
movies.first()

'movieId,title,genres'

In [20]:
ratings.first()

'userId,movieId,rating,timestamp'

In [21]:
movies_by_movieid =moviesData\
.map(lambda line: line.split(","))\
.map(lambda tokens: (int(tokens[0]),tokens[1]))

In [22]:
movies_by_movieid.first()

(1, 'Toy Story (1995)')

In [23]:
ratings_by_movieid = ratingsData\
.map(lambda line: line.split(","))\
.map(lambda tokens: (int(tokens[1]), float(tokens[2])))

ratings_by_movieid.first()

(31, 2.5)

In [24]:
top_10 = movies_by_movieid.join(ratings_by_movieid)\
.map(lambda p: ((p[0], p[1][0]), p[1][1]))\
.groupByKey().mapValues(lambda values: (sum(values)/len(values), len(values)))\
.filter(lambda p: p[1][1] >= 100)\
.sortBy(lambda p: p[1], False)\
.take(10)

for m in top_10:
    print(m)

((858, '"Godfather'), (4.4875, 200))
((318, '"Shawshank Redemption'), (4.487138263665595, 311))
((1221, '"Godfather: Part II'), (4.385185185185185, 135))
((50, '"Usual Suspects'), (4.370646766169155, 201))
((527, "Schindler's List (1993)"), (4.30327868852459, 244))
((1193, "One Flew Over the Cuckoo's Nest (1975)"), (4.256944444444445, 144))
((608, 'Fargo (1996)'), (4.256696428571429, 224))
((296, 'Pulp Fiction (1994)'), (4.256172839506172, 324))
((2858, 'American Beauty (1999)'), (4.236363636363636, 220))
((58559, '"Dark Knight'), (4.235537190082645, 121))
